<a href="https://colab.research.google.com/github/vinayvp/pose_estimation/blob/main/PRISM_Software.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##PRISM - SAMUSNG PROJECT CODE

###We are using AlphaPose for detecting/predicting the keypoints

####Cloning from git and then importing all the necessary dependencies

In [1]:
!git clone -b pytorch https://github.com/MVIG-SJTU/AlphaPose.git

Cloning into 'AlphaPose'...
remote: Enumerating objects: 2301, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 2301 (delta 23), reused 1 (delta 0), pack-reused 2245
Receiving objects: 100% (2301/2301), 108.48 MiB | 34.57 MiB/s, done.
Resolving deltas: 100% (1124/1124), done.


In [3]:
!echo "# pose_estimation" >> README.md

In [4]:
!git init

Reinitialized existing Git repository in /content/AlphaPose/.git/


In [5]:
!git add README.md

In [8]:
!git config --global user.email "vinaysurfing@gmail.com"
!git config --global user.name "vinayvp"

In [9]:
!git commit -m "first commit"
!git branch -M main
!git remote add origin https://github.com/vinayvp/pose_estimation.git
!git push -u origin main

[main c62b838] first commit
 1 file changed, 1 insertion(+)
fatal: remote origin already exists.
fatal: could not read Username for 'https://github.com': No such device or address


In [2]:
%cd AlphaPose/

/content/AlphaPose


In [ ]:
!pip install PyDrive visdom

     |████████████████████████████████| 676 kB 4.6 MB/s 
     |████████████████████████████████| 54 kB 2.3 MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-py3-none-any.whl size=655250 sha256=d7422549307f1051f8e8e10b9b14adc4643053b46c78362a912b43753a8859c1
  Stored in directory: /root/.cache/pip/wheels/2d/d1/9b/cde923274eac9cbb6ff0d8c7c72fe30a3da9095a38fd50bbf1
  Created wheel for torchfile: filename=torchfile-0.1.0-py3-none-any.whl size=5709 sha256=d1fa202db829570e1670f201ec2cb2fe4690800dba285a46ac15e2eac9980e14
  Stored in directory: /root/.cache/pip/wheels/ac/5c/3a/a80e1c65880945c71fd833408cd1e9a8cb7e2f8f37620bb75b
Successfully built visdom torchfile


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

Connect and allow permissions to access your drive
For importing these files -


1.   Pose Estimation Model - https://drive.google.com/file/d/1OPORTWB2cwd5YTVBX-NE8fsauZJWsrtW/view
2.   YOLO V3 Model - https://drive.google.com/file/d/1D47msNOOiJKvPOXlnpyzdKA3k6E97NTC/view



In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
#Importing Pose estimation model
downloaded = drive.CreateFile({'id':"1OPORTWB2cwd5YTVBX-NE8fsauZJWsrtW"})
downloaded.GetContentFile('/content/AlphaPose/models/sppe/duc_se.pth') 

#Importing YOLOV3 Modele
downloaded = drive.CreateFile({'id':"1D47msNOOiJKvPOXlnpyzdKA3k6E97NTC"})
downloaded.GetContentFile('/content/AlphaPose/models/yolo/yolov3-spp.weights')

###Frames Extraction Part - from videos

In [ ]:
import cv2
import time
import os

Set the Input folder of all the videos/clips (Import it from drive)

In [ ]:
# Add your folder path containing videos in this variable - path_source
path_source = 'D:/POse Estimation/Dataset/Jumping/'

# Add the folder you want to store your frames
path_dest = 'D:/POse Estimation/Dataset/Jumping_frames/'

# Define frame rate here
frame_rate = 10

In [ ]:
# get a list of all videos
videos_list = os.listdir(path_source)
print("Number of Total videos in this folder -",len(videos_list))

In [ ]:
def get_frames(frame_rate,path_s,path_d,file):
    cap = cv2.VideoCapture(path_s+file)
    prev = 0
    i = 0
    while cap.isOpened():
        time_elapsed = time.time() - prev
        ret, frame = cap.read()
        if not ret:
            break

        if time_elapsed > 1./frame_rate:
            prev = time.time()
            cv2.imwrite(path_d+'test_'+str(i)+'.jpg', frame)
            i += 1

    # print("Frames extracted: ",i)
    cap.release()
    cv2.destroyAllWindows()
    return i

Run this below cell to get your frames in the specified folder

In [ ]:
# cnt - stores total frames extracted from that folder
cnt = 0

# Run to get frames
x=1

for file in videos_list:
    i = get_frames(frame_rate,path_source,path_dest,file)
    print("Done ",x,"- frm.extracted =",i)
    x+=1
    cnt+=i
print("Total Frames extracted: ",cnt)

###Now we shall use AlphaPose model to get all the keypoints

In [ ]:
# Set the path of folder of frames you want ot annotate
path_input = '/content/AlphaPose/examples/demo'

# Set the path of folder for annotations to be saved
annotation_folder = '/content/AlphaPose/examples'

Run below cell to run the pose estimation model to identify 16-keypoints for the given images

In [ ]:
!python3 demo.py --indir /content/AlphaPose/examples/demo  --outdir annotation_folder --save_img --fast_inference false

Loading YOLO model..
Loading pose model from ./models/sppe/duc_se.pth
100% 3/3 [00:01<00:00,  2.88it/s]
===========================> Finish Model Running.
===========================> Rendering remaining images in the queue...
===========================> If this step takes too long, you can enable the --vis_fast flag to use fast rendering (real-time).
[W CudaIPCTypes.cpp:15] Producer process has been terminated before all shared CUDA tensors released. See Note [Sharing CUDA tensors]


###Pre-Processing the Annotations for importing into SuperAnnotate